# HDS5210-2024 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 24th.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [46]:
def priest(sex, age, rsp_rt, oxy_st, hrt_rt, systolic_bp, temp_c, alertness, inspire_oxy, pmf_status):
    """
    Calculate the PRIEST Clinical Severity Score for a patient.

    Args:
        sex(str) : Gender asigned at birth
        age(int): Age in years
        rsp_rt(int): Respiratory rate in breaths per minute
        oxy_st(float): Oxygen saturation as a percent between 0 and 1
        hrt_rt(int): Heart rate in beats per minute
        systolic_bp(int): Systolic BP in mmHg
        temp_c(float): Temperature in degrees Celsius
        alertness(str): Alertness status('alert','confused','not alert')
        inspire_oxy(str): Oxygen inspiration('air','supplemental oxygen')
        pmf_status(str): Performance Status ('unrestricted normal activity','limited strenuous activity, can do light activity','limited activity, can self-care','limited self-care')

    Returns:
       float: Risk percentage as a numeric value between 0 and 1

    >>> priest("male",68,12,0.95,85,110,37.5,'alert','air','unrestricted normal activity')
    0.15
    >>> priest('FEMALE', 80, 25, 0.92, 100, 100, 36.8, 'not alert', 'supplemental oxygen', 'bed/chair bound, no self-care')
    0.59
    >>> priest('female', 68, 23, 0.91, 40, 90, 34.0, 'confused', 'supplemental oxygen', 'bed/chair bound, no self-care')
    0.99
    """
    # Normalize strings to lowercase
    sex = sex.lower()
    alertness = alertness.lower()
    inspire_oxy = inspire_oxy.lower()
    pmf_status = pmf_status.lower()

    # Calculating points for sex variable
    if sex == 'male':
      sex_score = 1
    elif sex == 'female':
      sex_score = 0
    else:
      return None

    # Calculating points for age variable
    if age < 16:
        return None
    if age >= 16 and age <= 49:
        age_score = 0
    elif age >= 50 and age <= 65:
        age_score = 2
    elif age >= 66 and age <= 80:
        age_score = 3
    elif age > 80:
        age_score = 4

    # Calculating points for rsp_rt variable
    if rsp_rt < 9:
        rsp_rt_score = 3
    elif rsp_rt >= 9 and rsp_rt <= 11:
        rsp_rt_score = 1
    elif rsp_rt >= 12 and rsp_rt <= 20:
        rsp_rt_score = 0
    elif rsp_rt >= 21 and rsp_rt <= 24:
        rsp_rt_score = 2
    else:
        rsp_rt_score = 3

    # Calculating points for oxy_st variable
    if oxy_st > 0.95:
        oxy_st_score = 0
    elif oxy_st >= 0.94 and oxy_st <= 0.95:
        oxy_st_score = 1
    elif oxy_st >= 0.92 and oxy_st <= 0.93:
        oxy_st_score = 2
    else:
        oxy_st_score = 3

    # Calculating points for hrt_rt variable
    if hrt_rt < 41:
        hrt_rt_score = 3
    elif hrt_rt >= 41 and hrt_rt <= 50:
        hrt_rt_score = 1
    elif hrt_rt >= 51 and hrt_rt <= 90:
        hrt_rt_score = 0
    elif hrt_rt >= 91 and hrt_rt <= 110:
        hrt_rt_score = 1
    elif hrt_rt >= 111 and hrt_rt <= 130:
        hrt_rt_score = 2
    else:
        hrt_rt_score = 3

    # Calculating points for systolic_bp variable
    if systolic_bp < 91:
        systolic_bp_score = 3
    elif systolic_bp >= 91 and systolic_bp <= 100:
        systolic_bp_score = 2
    elif systolic_bp >= 101 and systolic_bp <= 110:
        systolic_bp_score = 1
    elif systolic_bp >= 111 and systolic_bp <= 219:
        systolic_bp_score = 0
    else:
        systolic_bp_score = 3

    # Calculating points for temp_c variable
    if temp_c < 35.1:
        temp_c_score = 3
    elif temp_c >= 35.1 and temp_c <= 36.0:
        temp_c_score = 1
    elif temp_c >= 36.1 and temp_c <= 38.0:
        temp_c_score = 0
    elif temp_c >= 38.1 and temp_c <= 39.0:
        temp_c_score = 1
    else:
        temp_c_score = 2

    # Calculating points for alertness variable
    if alertness not in ['alert', 'confused', 'not alert']:
      return None  # Return None if alertness input is invalid

    # Assign scores based on alertness status
    if alertness == 'alert':
      alertness_score = 0  # Score 0 for alert
    elif alertness == 'confused or not alert':
      alertness_score = 3
    elif alertness in ['confused', 'not alert']:
      alertness_score = 3  # Score 3 for confused or not alert
    # Calculating points for inspire_oxy variable
    if inspire_oxy not in ['air', 'supplemental oxygen']:
      return None  # Return None if inspired oxygen input is invalid

    # Assign scores based on inspired oxygen status
    if inspire_oxy == 'air':
      inspire_oxy_score = 0  # Score 0 for room air
    elif inspire_oxy == 'supplemental oxygen':
      inspire_oxy_score = 2  # Score 2 for supplemental oxygen

    # Calculating points for pmf_status variable
    if pmf_status not in ['unrestricted normal activity', 'limited strenuous activity, can do light activity', 'limited activity, can self-care', 'limited self-care', 'bed/chair bound, no self-care']:
        return None
    elif pmf_status == 'unrestricted normal activity':
        pmf_status_score = 0
    elif pmf_status == 'limited strenuous activity, can do light activity':
        pmf_status_score = 1
    elif pmf_status == 'limited activity, can self-care':
        pmf_status_score = 2
    elif pmf_status == 'limited self-care':
        pmf_status_score = 3
    elif pmf_status == 'bed/chair bound, no self-care':
        pmf_status_score = 4

    # Calculating the total score
    sum_of_score = (
        sex_score + age_score + rsp_rt_score + oxy_st_score + hrt_rt_score +
        systolic_bp_score + temp_c_score + alertness_score + inspire_oxy_score + pmf_status_score
    )

    # Calculating the risk percentage based on the total score
    if sum_of_score >= 0 and sum_of_score <= 1:
        risk_pct = 0.01
    elif sum_of_score >= 2 and sum_of_score <= 3:
        risk_pct = 0.02
    elif sum_of_score == 4:
        risk_pct = 0.03
    elif sum_of_score == 5:
        risk_pct = 0.09
    elif sum_of_score == 6:
        risk_pct = 0.15
    elif sum_of_score == 7:
        risk_pct = 0.18
    elif sum_of_score == 8:
        risk_pct = 0.22
    elif sum_of_score == 9:
        risk_pct = 0.26
    elif sum_of_score == 10:
        risk_pct = 0.29
    elif sum_of_score == 11:
        risk_pct = 0.34
    elif sum_of_score == 12:
        risk_pct = 0.38
    elif sum_of_score == 13:
        risk_pct = 0.46
    elif sum_of_score == 14:
        risk_pct = 0.47
    elif sum_of_score == 15:
        risk_pct = 0.49
    elif sum_of_score == 16:
        risk_pct = 0.55
    elif sum_of_score >= 17 and sum_of_score <= 25:
        risk_pct = 0.59
    elif sum_of_score > 25:
        risk_pct = 0.99
    else:
        return None

    return round(risk_pct, 2)

In [47]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest("male",68,12,0.95,85,110,37.5,'alert','air','unrestricted normal activity')
Expecting:
    0.15
ok
Trying:
    priest('FEMALE', 80, 25, 0.92, 100, 100, 36.8, 'not alert', 'supplemental oxygen', 'bed/chair bound, no self-care')
Expecting:
    0.59
ok
Trying:
    priest('female', 68, 23, 0.91, 40, 90, 34.0, 'confused', 'supplemental oxygen', 'bed/chair bound, no self-care')
Expecting:
    0.99
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [48]:
import requests

def find_hospital(age, sex, risk_pct):
    """
    Finds appropriate hospital for a patient based on age, sex, and risk percentage.

    - age (int): The age of the patient.
    - sex (str): The sex of the patient ('male' or 'female').
    - risk_pct (float): The risk percentage as a decimal.

    Returns:
    - str: The name of the recommended hospital, or None if the request fails.

    >>> find_hospital(40, 'male', 0.1)
    'Southwest Hospital and Medical Center'

    >>> find_hospital(16, 'male', 0.29)
    'Childrens Healthcare of Atlanta at Scottish Rite'

    >>> find_hospital(32, 'male', 0.8)
    'Emory Dunwoody Medical Center'

    >>> find_hospital(66, 'female', 0.34)
    'Wesley Woods Geriatric Hospital'

    """

    parent_url = "https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/"

    sex= sex.lower() #Converting the sex to lowercase

    parameters = [] # Making the query parameters using a list
    parameters.append("age=" + str(age))
    parameters.append("sex=" + sex)
    parameters.append("risk_pct=" + str(risk_pct))


    parameter_string = ""
    for i in range(len(parameters)):
        if i == 0:
            parameter_string += parameters[i]  # Add the first parameter
        else:
            parameter_string += "&" + parameters[i]  # Adding the subsequent parameters with '&'

    url = parent_url + "?" + parameter_string    # Making the full URL

    response = requests.get(url)

    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            return data.get('hospital')
        else:
            return None
    except:
        return None

In [49]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(40, 'male', 0.1)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(16, 'male', 0.29)
Expecting:
    'Childrens Healthcare of Atlanta at Scottish Rite'
ok
Trying:
    find_hospital(32, 'male', 0.8)
Expecting:
    'Emory Dunwoody Medical Center'
ok
Trying:
    find_hospital(66, 'female', 0.34)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [54]:
import requests

def get_address(hospital_name):
    """
    Retrieves the address of a hospital based on its name from a JSON document.

    Args:
        hospital_name (str): The name of the hospital to search for.

    Returns:
        str: The address of the hospital or None if not found.

    Test Cases:
    >>> get_address("Wesley Woods Geriatric Hospital")
    '1821 CLIFTON ROAD, NE'
    >>> get_address("HABERSHAM COUNTY MEDICAL CTR")
    '541 HISTORIC HIGHWAY 441-NORTH'
    >>> get_address("COFFEE REGIONAL MEDICAL CENTER")
    '1101 OCILLA ROAD'
    """

    json_url = "https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-"

    try:
        response = requests.get(json_url)

        if response.status_code == 200:
            hospitals_data = response.json()

            # Search for the hospital by name
            hospital_name_upper = hospital_name.upper()  # Convert input to uppercase
            if hospital_name_upper in hospitals_data:
                return hospitals_data[hospital_name_upper]['ADDRESS']  # Return the address

    except Exception as e:
        print(f"Error: {e}")

    return None  # Return None if the hospital was not found or an error occurred


In [55]:
import doctest
doctest.run_docstring_examples(get_address, globals(),verbose=True)

Finding tests in NoName
Trying:
    get_address("Wesley Woods Geriatric Hospital")
Expecting:
    '1821 CLIFTON ROAD, NE'
ok
Trying:
    get_address("HABERSHAM COUNTY MEDICAL CTR")
Expecting:
    '541 HISTORIC HIGHWAY 441-NORTH'
ok
Trying:
    get_address("COFFEE REGIONAL MEDICAL CENTER")
Expecting:
    '1101 OCILLA ROAD'
ok


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [43]:
import requests

def process_people(data):
    """
    Returns a dictionary containing lists of patients.

    Args:
    data: A string containing the URL to the data file.

    Returns:
    dict: A dictionary containing lists of patients.
    """
    data = "https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh"

    try:

        response = requests.get(data)

        if response.status_code == 200:
            data_new = response.text

        lines = data_new.splitlines() # Split the response text into lines

        patient_data = {}     # Initialize an empty dictionary to store patient data

        # Process each line after the header
        for line in lines[1:]:  # Skip the header line
            values = line.split('|')  # Split the line by the pipe character

            # Extract patient data
            patient_number = values[0]
            sex = values[1]
            age = int(values[2])
            rsp_rt = float(values[3])
            oxy_st = float(values[4])
            hrt_rt = float(values[5])
            systolic_bp = float(values[6])
            temp_c = float(values[7])
            alertness = values[8]
            inspire_oxy = values[9]
            pmf_status = values[10]

            risk = priest(sex, age, rsp_rt, oxy_st, hrt_rt, systolic_bp, temp_c, alertness, inspire_oxy, pmf_status) # Value of risk_score using priest function

            # If the risk is valid, find the hospital and address
            if risk is not None:
                hospital = find_hospital(age, sex, risk)  # Find the hospital based on age, sex, and risk
                if hospital:
                    address = get_address(hospital)  # Get the address if a hospital is found
                else:
                    address = None  # Set address to None if no hospital is found

                # Store the results in the dictionary and add risk,hospital and address
                patient_data[patient_number] = [sex, age,rsp_rt,oxy_st,hrt_rt,systolic_bp, temp_c,alertness,inspire_oxy,pmf_status,risk, hospital,address]

        return patient_data  # Return the dictionary containing patient data

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [44]:
process_people('https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh')

{'E9559': ['FEMALE',
  40,
  24.0,
  0.96,
  105.0,
  115.0,
  34.9,
  'ALERT',
  'AIR',
  'unrestricted normal activity',
  0.15,
  'Select Specialty Hospital - Northeast Atlanta',
  '1821 CLIFTON ROAD NE'],
 'E9385': ['Female',
  51,
  19.0,
  0.99,
  90.0,
  91.0,
  38.1,
  'ALERT',
  'supplemental oxygen',
  'unrestricted normal activity',
  0.18,
  'Select Specialty Hospital - Northeast Atlanta',
  '1821 CLIFTON ROAD NE'],
 'E3067': ['female',
  40,
  29.0,
  0.96,
  105.0,
  95.0,
  38.1,
  'ALERT',
  'air',
  'unrestricted normal activity',
  0.18,
  'Select Specialty Hospital - Northeast Atlanta',
  '1821 CLIFTON ROAD NE'],
 'E9422': ['FEMALE',
  66,
  19.0,
  0.96,
  135.0,
  115.0,
  38.1,
  'ALERT',
  'AIR',
  'bed/chair bound, no self-care',
  0.34,
  'Wesley Woods Geriatric Hospital',
  '1821 CLIFTON ROAD, NE'],
 'E6235': ['Male',
  16,
  23.0,
  0.96,
  135.0,
  115.0,
  39.2,
  'ALERT',
  'supplemental oxygen',
  'unrestricted normal activity',
  0.29,
  'Childrens Healt

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [45]:
import requests
import json

# URL for the expected results
results_url = "https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5"

# Fetch the expected results
response = requests.get(results_url)
if response.status_code == 200:
    expected_results = response.json()  # Load JSON data
else:
    print(f"Failed to retrieve expected results: {response.status_code}")

# Process the patient data
patient_results = process_people('https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh')

# Check for discrepancies
if patient_results:
    for patient_number, patient_info in patient_results.items():
        if patient_number in expected_results:
            # Compare individual fields
            if patient_info != expected_results[patient_number]:
                print(f"Discrepancy found for patient {patient_number}:")
                print(f"Your result: {patient_info}")
                print(f"Expected result: {expected_results[patient_number]}\n")
        else:
            print(f"Patient {patient_number} not found in expected results.")

    # Check for expected patients that are missing from your results
    for patient_number in expected_results:
        if patient_number not in patient_results:
            print(f"Expected patient {patient_number} is missing from your results.")
else:
    print("No patient results available to compare.")

Expected patient E8661 is missing from your results.
Expected patient E8800 is missing from your results.
Expected patient E7110 is missing from your results.
Expected patient E1700 is missing from your results.
Expected patient E2117 is missing from your results.
Expected patient E6885 is missing from your results.
Expected patient E4564 is missing from your results.
Expected patient E1904 is missing from your results.
Expected patient E1073 is missing from your results.
Expected patient E2001 is missing from your results.
Expected patient E3739 is missing from your results.
Expected patient E8560 is missing from your results.
Expected patient E5361 is missing from your results.


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---